In [126]:
import ast
import pandas as pd
from datasets import load_dataset

In [127]:
# Load the 'recetasdelaabuela' dataset from Hugging Face
dataset = load_dataset("somosnlp/recetasdelaabuela", 'version_inicial')
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Display the dataset structure
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Nombre', 'URL', 'Ingredientes', 'Pasos', 'Pais', 'Duracion', 'Porciones', 'Calorias', 'Categoria', 'Contexto', 'Valoracion y Votos', 'Comensales', 'Tiempo', 'Dificultad', 'Categoria 2', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],
        num_rows: 20085
    })
})


### Data Cleaning

In [128]:
# convert all column names to lowercase
df.columns = df.columns.str.lower()

# Drop 'Id' column and rename 'index' to Id
df.drop(columns=['id'], inplace=True)
df.reset_index(drop= True, inplace=True)

# convert Dificultidad column from 1 to 5 where muy baja is 1 and muy alta is 5. using a dictionary
dificultidad_map = {
    'muy baja': 1,
    'baja': 2,
    'media': 3,
    'alta': 4,
    'muy alta': 5
}
df['dificultad'] = df['dificultad'].str.replace('Dificultad ','').map(dificultidad_map)


In [ ]:
# name for a function to convert any column from a string representation of a list to a space-separated string.
def convert_list_to_string(input_string):
    """
    Convert a column from a string representation of a list to a space-separated string.
    """
    if isinstance(input_string, str) and '[' in input_string:
        # Convert the string representation of a list to an actual list
        try:
            return ' '.join(ast.literal_eval(input_string))
        except (ValueError, SyntaxError):
            # If conversion fails, pass and return the original value
            pass
    return input_string      

def convert_valoracion_y_votos(input_string):
    """
    Convert the 'Valoración y votos' column to separate 'valoracion' and 'votos' columns.
    Example:
    >>> convert_valoracion_y_votos('Valoración: 4,5 (2 votos)')
    0    4.5
    1    2.0
    dtype: float64
    """
    if isinstance(input_string, str) and ':' in input_string:
        # Split the string into valoracion and votos
        try:
            valoracion, votos = input_string.split(':')[1].split('(')
            # Extract the numeric values
            valoracion = float(valoracion.split(' ')[1].replace(',', '.'))
            votos = int(votos.split(' ')[0])
            return pd.Series([valoracion, votos])
        except (ValueError, SyntaxError):
            # If conversion fails, pass and return None
            pass
    return pd.Series([None, None])  # Return None for both columns if conversion fails                                

In [ ]:
df['pasos'] = df['pasos'].apply(lambda ss: convert_list_to_string(ss)) # Convert the 'Pasos' column
df['ingredientes'] = df['ingredientes'].apply(lambda ss: convert_list_to_string(ss)) # Convert the 'Ingredientes' column

# convert 'volaracion y votos' column to two separate columns
df[['valoracion', 'votos']] = df['valoracion y votos'].apply(lambda ss: convert_valoracion_y_votos(ss))

# edit the 'comensales' column to only keep the number of guests
df['comensales'] = df['comensales'].apply(lambda ss: int(ss.split(' ')[0]) if ss is not None else None) # 


In [176]:
# country Mexico and volaracion y votos is not None
df[(df['pais'] == 'Guatemala')].head()

,nombre,url,ingredientes,pasos,pais,duracion,porciones,calorias,categoria,contexto,valoracion y votos,comensales,tiempo,dificultad,categoria 2,unnamed: 16,unnamed: 17,unnamed: 18,valoracion,votos
2756,quiche de puerros,https://www.recetasgratis.net/receta-de-quiche...,1 lámina de hojaldre 2 puerros 3 huevos 100 mi...,1 Limpiamos los puerros dejando solo la parte ...,Guatemala,45m,"['1 lámina de hojaldre', '2 puerros', '3 huevo...",None,"['Recomendada para vegetarianos', 'Recetas al ...","Quiché de puerro, una tarta salada de verdura ...","Valoración: 3,6 (21 votos)",4.0,Entrante,2.0,"Bajo en calorías, Sin grasa, Alto en grasas sa...",None,None,quiche de puerros,3.6,21.0
2932,ceviche guatemalteco de camaron,https://www.recetasgratis.net/receta-de-cevich...,½ kilogramo de Camarones 5 unidades de Tomate ...,1 Lo primero que debes hacer para poder realiz...,Guatemala,24h,"['½ kilogramo de Camarones', '5 unidades de To...",None,[],El ceviche es uno de los apertivos más típicos...,"Valoración: 4,1 (88 votos)",4.0,Entrante,2.0,"Bajo en calorías, Sin grasa, Alto en fibra",None,None,ceviche guatemalteco de camaron,4.1,88.0
3114,fiambre aleman paso a paso,https://www.recetasgratis.net/receta-de-fiambr...,12 panqueques 2 tomates 2 huevos duros 200 gra...,1 Preparar todos los ingredientes antes de ini...,Guatemala,30m,"['12 panqueques', '2 tomates', '2 huevos duros...",None,[],Disfruta de esta receta de fiambre alemán paso...,"Valoración: 4,2 (5 votos)",8.0,Entrante,2.0,"Bajo en calorías, Sin grasa, Alto en grasas sa...",None,None,fiambre aleman paso a paso,4.2,5.0
3240,avena horneada,https://www.recetasgratis.net/receta-de-avena-...,1 taza de avena triturada 2 cucharadas de post...,1 ¿Te preguntas cómo hacer avena al horno? Pue...,Guatemala,30m,"['1 taza de avena triturada', '2 cucharadas de...",None,['Recetas al horno'],Si ya estás cansado de los desayunos aburridos...,None,5.0,Desayuno,2.0,"Bajo en calorías, Sin grasa, Alto en fibra",None,None,avena horneada,NaN,NaN
5993,leche con platano,https://www.recetasgratis.net/receta-de-leche-...,1 tasa de leche 3 platanos 1 cuchara de asucar...,1 En la jugera bate todo junto y despues lo ec...,Guatemala,None,"['1 tasa de leche', '3 platanos', '1 cuchara d...",None,None,None,"Valoración: 3,7 (92 votos)",NaN,None,4.0,"Bajo en calorías, Sin grasa, Alto contenido de...",None,None,leche con platano,3.7,92.0


In [ ]:
# drop volaracion y votos column
df.drop(columns=['valoracion y votos'], inplace=True)

#### Further Data Cleaning Steps TODO

* Convert the `duracion` column to minutes from various formats (e.g., "45 min", "1h 30m").
* Handle multiple values in the `categoria` and `categoria 2` columns by splitting them into separate rows or creating lists.